In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from scipy.stats import zscore

# HoloViz and Bokeh
import colorcet as cc
import holoviews as hv; hv.extension('bokeh')
from holoviews.plotting.links import RangeToolLink
from holoviews.operation.datashader import rasterize
from holoviews import Dataset
from bokeh.models import HoverTool, WheelZoomTool
import panel as pn; pn.extension(template='material')

# Neuro repo
from neurodatagen.eeg import generate_eeg_powerlaw
from hvneuro import download_file

In [ ]:
%%time
n_channels = 25
n_seconds = 30
fs = 512

data, time, channels = generate_eeg_powerlaw(n_channels, n_seconds, fs, channel_prefix='EEG')

print(f'shape: {data.shape} (n_channels, samples) ')
data

In [ ]:
max_ch_disp = 10  # max channels to initially display
max_t_disp = 5 # max time in seconds to initially display

annotation = hv.VSpan(19, 20).opts(color='yellow', alpha=.15) # example annotation (start, end) time

hover = HoverTool(tooltips=[
    ("Channel", "@channel"),
    ("Time", "$x s"),
    ("Amplitude", "$y µV")])

In [ ]:
channel_curves = []
for channel, channel_data in zip(channels, data):
    ds = Dataset((time, channel_data, channel), ["Time", "Amplitude", "channel"])
    curve = hv.Curve(ds, "Time", ["Amplitude", "channel"], label=f'{channel}')
    curve.opts(color="black", line_width=1, subcoordinate_y=True, tools=[hover])
    channel_curves.append(curve)

eeg_viewer = (annotation * hv.Overlay(channel_curves, kdims="Channel"))
eeg_viewer = eeg_viewer.opts(xlabel="Time (s)", ylabel="Channel", show_legend=False, aspect=3, responsive=True)
    # padding=0, xlabel="Time (s)", ylabel="Channel",
    # show_legend=False, aspect=2, responsive=True,
    # shared_axes=False, xlim=(time.min(), time.max()), backend_opts={
    #     "x_range.bounds": (time.min(), time.max()),
    #     "y_range.bounds": (0, len(channels))})

y_positions = range(len(channels))
yticks = [(i, ich) for i, ich in enumerate(channels)]

z_data = zscore(data, axis=1)

minimap = rasterize(hv.Image((time, y_positions, z_data), ["Time (s)", "Channel"], "Amplitude (uV)"))

clim_mul = 3
minimap = minimap.opts(
    cmap="RdBu_r", colorbar=False, xlabel='', alpha=.5, yticks=[yticks[0], yticks[-1]],
    height=125, responsive=True, default_tools=[], shared_axes=False, clim=(-z_data.std()*clim_mul, z_data.std()*clim_mul))

# Create RangeToolLink between the minimap and the main EEG viewer 
max_y_disp = np.min([len(channels)+1, max_ch_disp+1])
RangeToolLink(minimap, eeg_viewer, axes=["x", "y"],
              boundsx=(None, max_t_disp),
              boundsy=(None, max_y_disp))

# eeg_app = (eeg_viewer + minimap*annotation).opts(merge_tools=False).cols(1)

# eeg_app = pn.Column((eeg_viewer + minimap * annotations_overlay).cols(1), min_height=650)
eeg_app = pn.Column((eeg_viewer + minimap * annotation).cols(1), min_height=650).servable(target='main', title='EEG Viewer with HoloViz and Bokeh')
eeg_app